# Simple Neighbourhood Approach (User-Based CF)
As a first step, we will use basic neighbourhood-based collaborative filtering (CF) techniques, with a simple model as a baseline.

### Pre-Processing

In [1]:
%%capture
import sys
import os

# Add project root to Python path
project_root = os.path.abspath("..")
if project_root not in sys.path:
    sys.path.append(project_root)
# import packages
from utils.imports import *
from models.UserBasedCF import UserBasedCF

In [2]:
# import pandas dataframes
with open("../data/dataframes.pkl", "rb") as f:
    data = pickle.load(f)

train = data["train"]
validation = data["validation"]
baseline = data["baseline"]

# load sparse matrix
ui_csr = load_npz("../data/ui_csr.npz")

# load encodings
with open("../artifacts/user_encoder.pkl", "rb") as f:
    user_encoder = pickle.load(f)
with open("../artifacts/item_encoder.pkl", "rb") as f:
    item_encoder = pickle.load(f)
with open("../artifacts/user_map.pkl", "rb") as f:
    user_map = pickle.load(f)
with open("../artifacts/item_map.pkl", "rb") as f:
    item_map = pickle.load(f)

## Model Fitting


In [3]:
# load and fit model
user_CF = UserBasedCF(ui_csr, validation, user_encoder, item_encoder)
user_CF.fit()

#### Evaluating the effect of neighbourhood size
Now we'll evaluate over different choices of k.

In [4]:
# let's experiment with different values of k
k_values = [3, 5, 10, 20, 50, 100]
for k in k_values:
    RMSE, MAE = user_CF.evaluate_error(k)
    print(f'The RMSE for user-based CF with {k}-NN is '
          f'{RMSE}')
    print(f'The MAE for user-based CF with {k}-NN is '
          f'{MAE}')
    print('\n')

The RMSE for user-based CF with 3-NN is 0.8281375351604974
The MAE for user-based CF with 3-NN is 0.6062849787578912


The RMSE for user-based CF with 5-NN is 0.7948962457823016
The MAE for user-based CF with 5-NN is 0.5767247519566189


The RMSE for user-based CF with 10-NN is 0.7731675049216724
The MAE for user-based CF with 10-NN is 0.555877013054669


The RMSE for user-based CF with 20-NN is 0.766080997095158
The MAE for user-based CF with 20-NN is 0.5465076312264295


The RMSE for user-based CF with 50-NN is 0.7604301674859361
The MAE for user-based CF with 50-NN is 0.5415636400772108


The RMSE for user-based CF with 100-NN is 0.7582446812298936
The MAE for user-based CF with 100-NN is 0.5399732653019377




#### Top-N predictions
We're not only interested in prediction accuracy; we'd also like to know how effective our algorithm is at predicting novel or less popular items. To measure this, we'll look at the top-N items as calculated by taking the $N$ items for a user with the highest predicted ratings. To avoid interminable runtime, we'll use a a fast matrix-based function which precomputes the most similar neighbours for each user (not for a user-item pair).


In [5]:
k_values = [3, 5, 10, 20]
N = 10

for k in k_values:
    cov = user_CF.coverage_at_N(k, N)
    print(f'For k = {k} nearest neighbours and top-{N} beers recommended:')
    print(f'The number of recommended beers over all users is {cov[1]}')
    print(f'This corresponds to {cov[0]}% '
        'of all beers in the training matrix.\n')

For k = 3 nearest neighbours and top-10 beers recommended:
The number of recommended beers over all users is 6514
This corresponds to 24.95% of all beers in the training matrix.

For k = 5 nearest neighbours and top-10 beers recommended:
The number of recommended beers over all users is 6140
This corresponds to 23.52% of all beers in the training matrix.

For k = 10 nearest neighbours and top-10 beers recommended:
The number of recommended beers over all users is 5041
This corresponds to 19.31% of all beers in the training matrix.

For k = 20 nearest neighbours and top-10 beers recommended:
The number of recommended beers over all users is 3813
This corresponds to 14.6% of all beers in the training matrix.



It's worth mentioning that this experiment does not tell the entire story when it comes to recommending items from a catalog. This simple user-based CF model is only capable of recommending beers which have been reviewed at least three times - this already excludes the many beers which have only been reviewed once or twice. This is in addition to the fact that the system is only likely to recommend a fraction of the total available beers as demonstrated by the above code. We can investigate how recommendations differ given different neighbourhood sizes.

In [6]:
def get_beer_data(beer_data, beer_ids):
    results = beer_data[beer_data['beer_beerid'].isin(beer_ids)].groupby([
        'beer_beerid', 'beer_name', 'brewery_name', 'beer_style'], group_keys=False, as_index=False).agg(
            {'review_overall': ['mean', 'count', 'std'],
            'beer_abv': ['mean']}
        ).sort_values(by=('review_overall', 'count'), ascending=False).set_index("beer_beerid")
    return results

In [7]:
# set params
user, k, N = 69, 3, 10
get_beer_data(baseline, user_CF.top_N_beers(user, k, N))

beer_name  \
                                                         
beer_beerid                                              
276                             Sierra Nevada Pale Ale   
665                                 Anchor Liberty Ale   
3842                            Trappistes Rochefort 6   
21505                               Lammin Kataja Olut   
21493                           Flensburger Winterbock   
21498        Dog & Pony Double Dry-hopped Imperial IPA   
21502                                 Dusty Trail Pale   
21499                                            Julöl   
21515                                  Haake Beck Pils   
21514                                 Premium Pilsener   

                                     brewery_name  \
                                                    
beer_beerid                                         
276                     Sierra Nevada Brewing Co.   
665                        Anchor Brewing Company   
3842                       Brasserie de Rochefort   
21505                             Lammin Sahti Oy   
21493        Flensburger Brauerei GmbH Und Co. KG   
21498            Maritime Pacific Brewing Company   
21502                             Amnesia Brewing   
21499                         Grebbestad Bryggeri   
21515                         Brauerei Beck & Co.   
21514                    Brauerei Herrenhausen KG   

                                 beer_style review_overall                  \
                                                      mean count       std   
beer_beerid                                                                  
276                 American Pale Ale (APA)       4.248130  2406  0.529726   
665                 American Pale Ale (APA)       4.096070  1374  0.540642   
3842                Belgian Strong Dark Ale       4.138889   756  0.511350   
21505                                 Sahti       3.150485   103  0.756783   
21493                                  Bock       3.500000    10  0.408248   
21498        American Double / Imperial IPA       3.900000     5  0.223607   
21502               American Pale Ale (APA)       3.300000     5  0.670820   
21499                          Vienna Lager       3.625000     4  0.478714   
21515                       German Pilsener       3.250000     4  0.645497   
21514                       German Pilsener       3.500000     3  0.000000   

            beer_abv  
                mean  
beer_beerid           
276              5.6  
665              6.0  
3842             7.5  
21505            7.0  
21493            7.0  
21498            7.5  
21502            5.2  
21499            5.3  
21515            4.9  
21514            4.9

In [8]:
# set params
user, k, N = 69, 20, 10
get_beer_data(baseline, user_CF.top_N_beers(user, k, N))

beer_name  \
                                                
beer_beerid                                     
1093                          Two Hearted Ale   
276                    Sierra Nevada Pale Ale   
14916                              Hop Wallop   
1118                          Chocolate Stout   
18862                            Burton Baton   
6947         Cuvée Van De Keizer Blauw (Blue)   
2233                        Summit Winter Ale   
3646                  Urthel Hibernus Quentum   
27265                       Bell's Wheat Love   
35405              Victor's MemoriAle Altbier   

                                brewery_name                      beer_style  \
                                                                               
beer_beerid                                                                    
1093                    Bell's Brewery, Inc.                    American IPA   
276                Sierra Nevada Brewing Co.         American Pale Ale (APA)   
14916                Victory Brewing Company  American Double / Imperial IPA   
1118                              Rogue Ales                  American Stout   
18862                   Dogfish Head Brewery  American Double / Imperial IPA   
6947                     Brouwerij Het Anker         Belgian Strong Dark Ale   
2233                  Summit Brewing Company                   Winter Warmer   
3646         De Leyerth Brouwerijen (Urthel)                          Tripel   
27265                   Bell's Brewery, Inc.                       Wheatwine   
35405           Two Brothers Brewing Company                         Altbier   

            review_overall                 beer_abv  
                      mean count       std     mean  
beer_beerid                                          
1093              4.320482  2529  0.506680      7.0  
276               4.248130  2406  0.529726      5.6  
14916             3.987629  1738  0.606045      8.5  
1118              4.115407  1733  0.592690      6.0  
18862             4.010145  1380  0.557992     10.0  
6947              4.145920   723  0.594126     11.0  
2233              3.791489   235  0.494772      6.1  
3646              4.038462   221  0.543236      9.0  
27265             3.983553   152  0.547171      7.7  
35405             4.095238    42  0.508922      7.8

In [9]:
# set params
user, k, N = 420, 3, 10
get_beer_data(baseline, user_CF.top_N_beers(user, k, N))

beer_name  \
                                                    
beer_beerid                                         
226          Great Lakes Edmund Fitzgerald Porter   
142                          Ommegang (Abbey Ale)   
1385                             Delirium Tremens   
228                   Great Lakes Dortmunder Gold   
773                                    Goudenband   
62328                Estate Homegrown Wet Hop Ale   
38366                   Samuel Adams Dunkelweizen   
67262                     Longshot Blackened Hops   
67267                      Longshot Friar Hop Ale   
38224                           Point Oktoberfest   

                                   brewery_name                 beer_style  \
                                                                             
beer_beerid                                                                  
226                 Great Lakes Brewing Company            American Porter   
142                            Brewery Ommegang                     Dubbel   
1385                           Brouwerij Huyghe    Belgian Strong Pale Ale   
228                 Great Lakes Brewing Company  Dortmunder / Export Lager   
773                          Brouwerij Liefmans         Flanders Oud Bruin   
62328                 Sierra Nevada Brewing Co.               American IPA   
38366        Boston Beer Company (Samuel Adams)               Dunkelweizen   
67262        Boston Beer Company (Samuel Adams)         American Black Ale   
67267        Boston Beer Company (Samuel Adams)                Belgian IPA   
38224                     Stevens Point Brewery       Märzen / Oktoberfest   

            review_overall                 beer_abv  
                      mean count       std     mean  
beer_beerid                                          
226               4.322813  1600  0.466191     5.80  
142               4.040414  1497  0.591441     8.50  
1385              4.022912  1353  0.589786     8.50  
228               4.290899   868  0.499479     5.80  
773               4.133333   465  0.576541     8.00  
62328             4.134085   399  0.441493     6.70  
38366             3.680000   275  0.553159     5.10  
67262             3.924731   186  0.467577     7.00  
67267             3.591892   185  0.561141     9.00  
38224             3.698718    78  0.560263     5.15

In [10]:
# set params
user, k, N = 420, 20, 10
get_beer_data(baseline, user_CF.top_N_beers(user, k, N))

beer_name  \
                                                        
beer_beerid                                             
131                     Ayinger Celebrator Doppelbock   
2751                           Racer 5 India Pale Ale   
226              Great Lakes Edmund Fitzgerald Porter   
10325           Péché Mortel (Imperial Stout Au Cafe)   
646                         Westmalle Trappist Tripel   
19216                    Oak Aged Yeti Imperial Stout   
11922                                       Titan IPA   
25755            Heavy Seas - Loose Cannon (Hop3 Ale)   
5428         New Holland Dragon's Milk Oak Barrel Ale   
1287                                    Bell's Porter   

                                                  brewery_name  \
                                                                 
beer_beerid                                                      
131          Privatbrauerei Franz Inselkammer KG / Brauerei...   
2751                                 Bear Republic Brewing Co.   
226                                Great Lakes Brewing Company   
10325                                   Brasserie Dieu Du Ciel   
646                                        Brouwerij Westmalle   
19216                             Great Divide Brewing Company   
11922                             Great Divide Brewing Company   
25755                                          Heavy Seas Beer   
5428                               New Holland Brewing Company   
1287                                      Bell's Brewery, Inc.   

                                   beer_style review_overall                  \
                                                        mean count       std   
beer_beerid                                                                    
131                                Doppelbock       4.293592  2013  0.513722   
2751                             American IPA       4.229022  1871  0.472854   
226                           American Porter       4.322813  1600  0.466191   
10325        American Double / Imperial Stout       4.264685  1396  0.528709   
646                                    Tripel       4.196698  1393  0.549363   
19216                  Russian Imperial Stout       4.082671  1385  0.551066   
11922                            American IPA       4.137327  1227  0.494349   
25755                            American IPA       4.073559  1006  0.473220   
5428                           American Stout       3.762470   842  0.654713   
1287                          American Porter       3.985976   820  0.517801   

            beer_abv  
                mean  
beer_beerid           
131             6.70  
2751            7.00  
226             5.80  
10325           9.50  
646             9.50  
19216           9.50  
11922           7.10  
25755           7.25  
5428           10.00  
1287            5.60

We can see clearly that increasing the neighbourhood size tends to recommend more popular beers. This effect is commonly observed when implementing user-based CF. We can think critically about why this may be. When generating recommendations from a small number of neighbours, we only take the recommendations from the most similar users - and their "votes" towards recommendations have a very large impact on the predicted items. Therefore, a more rare and polarizing beer may be recommended to a user when using a small value for k if that user's closest neighbours give it a high score. Conversely, including a larger number of neighbours when generating predictions tends to push predicted scores towards the mean; more rare and polarizing beers will tend to get pushed out by popular and highly-rated beers. The "signal" from similar users will be lost. As we saw from the EDA, there are a relatively small number of highly-rated and highly-reviewed beers. Continuing to increase the value of k will lead to those beers being recommended:

In [11]:
# set params, make k big
user, k, N = 420, 500, 10
get_beer_data(baseline, user_CF.top_N_beers(user, k, N))

beer_name  \
                                                                 
beer_beerid                                                      
131                              Ayinger Celebrator Doppelbock   
2751                                    Racer 5 India Pale Ale   
141                                Hennepin (Farmhouse Saison)   
3457         Three Philosophers Belgian Style Blend (Quadru...   
226                       Great Lakes Edmund Fitzgerald Porter   
16403                             Smuttynose IPA "Finest Kind"   
10325                    Péché Mortel (Imperial Stout Au Cafe)   
6518                                           Dale's Pale Ale   
19216                             Oak Aged Yeti Imperial Stout   
11922                                                Titan IPA   

                                                  brewery_name  \
                                                                 
beer_beerid                                                      
131          Privatbrauerei Franz Inselkammer KG / Brauerei...   
2751                                 Bear Republic Brewing Co.   
141                                           Brewery Ommegang   
3457                                          Brewery Ommegang   
226                                Great Lakes Brewing Company   
16403                               Smuttynose Brewing Company   
10325                                   Brasserie Dieu Du Ciel   
6518                                  Oskar Blues Grill & Brew   
19216                             Great Divide Brewing Company   
11922                             Great Divide Brewing Company   

                                   beer_style review_overall                  \
                                                        mean count       std   
beer_beerid                                                                    
131                                Doppelbock       4.293592  2013  0.513722   
2751                             American IPA       4.229022  1871  0.472854   
141                    Saison / Farmhouse Ale       4.243311  1794  0.509828   
3457                         Quadrupel (Quad)       3.981173  1620  0.564645   
226                           American Porter       4.322813  1600  0.466191   
16403                            American IPA       4.124649  1424  0.577223   
10325        American Double / Imperial Stout       4.264685  1396  0.528709   
6518                  American Pale Ale (APA)       4.070605  1388  0.544409   
19216                  Russian Imperial Stout       4.082671  1385  0.551066   
11922                            American IPA       4.137327  1227  0.494349   

            beer_abv  
                mean  
beer_beerid           
131              6.7  
2751             7.0  
141              7.7  
3457             9.8  
226              5.8  
16403            6.9  
10325            9.5  
6518             6.5  
19216            9.5  
11922            7.1

All of these beers are among the most popular and well-liked beers on the site. This demonstrates why it's not beneficial to choose a large value of k simply because it results in a low RMSE; in the context of an item catalog, a recommender should be capable of providing personalized and novel recommendations.